# 2D SSN Model

1. Imports

In [1]:
import jax.numpy as np
import jax
import matplotlib.pyplot as plt
import time, os, json
import pandas as pd
from scipy import stats 
from tqdm import tqdm
import seaborn as sns
from jax import random

from SSN_classes_jax import SSN2DTopoV1_AMPAGABA_ONOFF
from util import GaborFilter, BW_Grating, find_A, create_gabor_filters

2. Create SSN network

In [2]:
#Network parameters
class ssn_pars():
    n = 2
    k = 0.04
    tauE = 30 # in ms
    tauI = 10 # in ms
    psi = 0.774
    tau_s = np.array([5, 7, 100]) #in ms, AMPA, GABA, NMDA current decay time constants
    


#Grid parameters
class grid_pars():
    gridsize_Nx = 9 # grid-points across each edge # gives rise to dx = 0.8 mm
    gridsize_deg = 2 * 1.6 # edge length in degrees
    magnif_factor = 2  # mm/deg
    hyper_col = 0.8 # mm   
    sigma_RF = 0.4 # deg (visual angle)

# Caleb's params for the full (with local) model:
Js0 = [1.82650658, 0.68194475, 2.06815311, 0.5106321]
gE, gI = 0.57328625, 0.26144141

sigEE, sigIE = 0.2, 0.40
sigEI, sigII = .09, .09

conn_pars = dict(
    PERIODIC = False,
    p_local = [.4, 0.7], # [p_local_EE, p_local_IE],
    sigma_oris = 1000) # sigma_oris



make_J2x2 = lambda Jee, Jei, Jie, Jii: np.array([[Jee, -Jei], [Jie,  -Jii]]) * np.pi * ssn_pars.psi
J_2x2 = make_J2x2(*Js0)
s_2x2 = np.array([[sigEE, sigEI],[sigIE, sigII]])

#Create network
ssn = SSN2DTopoV1_AMPAGABA_ONOFF(ssn_pars, grid_pars, conn_pars=conn_pars, J_2x2=J_2x2, s_2x2=s_2x2)


3. Create Gabor filters

In [3]:
#Gabor parameters 
sigma_g= 0.5
k= np.pi/(6*sigma_g)
general_pars = dict(k=k, edge_deg=3.2,  degree_per_pixel=0.05) #parameters shared with input stimuli


#Create filters
SSN_filters, A =create_gabor_filters(ssn, sigma_g=sigma_g, conv_factor = grid_pars.magnif_factor, **general_pars)

Average A is 0.0008299059715404468


3. Input target and reference

In [5]:
#Stimuli parameters
stimuli_pars = dict(outer_radius=3, inner_radius=2.5, grating_contrast=0.99)
stimuli_pars.update(general_pars)

#Create reference stimuli
ori_ref = 0
ref_grating=BW_Grating(ori_deg = ori_ref, **stimuli_pars).BW_image()

#Create target stimuli
ori_target= 10
target_grating=BW_Grating(ori_deg = ori_target, **stimuli_pars).BW_image()

# MODEL TRAINING

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def binary_loss(n, x):
    return n*np.log(x) + (1-n)*np.log(1-x)

def model(J_2x2, s_2x2, ssn_pars, grid_pars, conn_pars, general_pars, label, ref, target, **conv_pars):
    
    #Initialise network
    ssn=SSN2DTopoV1_AMPAGABA_ONOFF(ssn_pars, grid_pars, conn_pars=conn_pars, J_2x2=J_2x2, s_2x2=s_2x2)
   
    #Create Gabor filters for network
    SSN_filters, A =create_gabor_filters(ssn, sigma_g=sigma_g, conv_factor = grid_pars.magnif_factor, **general_pars)                               
                                       
    #Apply Gabor filters to stimuli
    output_ref=np.matmul(SSN_filters, ref.ravel())*A
    output_target=np.matmul(SSN_filters, target.ravel())*A
    
    #Rectify output
    SSN_input_ref=np.maximum(0, output_ref)
    SSN_input_target=np.maximum(0, output_target)
    
    #Input to SSN
    r_init = np.zeros(SSN_input_ref.shape[0])
    
    fp_ref, _ = ssn.fixed_point_r(SSN_input_ref, r_init=r_init, **conv_pars)
    
    fp_target, _ = ssn.fixed_point_r(SSN_input_target, r_init=r_init, **conv_pars)
    
    #Combine reference and target 
    x = fp_ref + fp_target
    
    #Apply sigmoid function
    x = sigmoid(0.1*x)
    
    #Calculate binary cross entropy loss
    loss=np.sum(binary_loss(label, x))
   
    #check what indices are nan
    #indices = np.argwhere(np.isnan(loss))
    
    return loss
    

def train_SSN(J_2x2, s_2x2, ssn_pars, grid_pars, conn_pars, general_pars, train_data, stimuli_pars, conv_pars):
    
    #Initialize loss
    total_loss = 0
    
       
    #find label
    if train_data[0] > train_data[1]:
        label=1
    else:
        label=0
        
    ref_grating=BW_Grating(ori_deg = train_data[0], **stimuli_pars).BW_image()
    target_grating = BW_Grating(ori_deg = train_data[1], **stimuli_pars).BW_image()
        
        #total_loss+=model(ssn, A, SSN_filters, label, ref_grating, target_grating, **conv_pars )
    
    grad_loss=jax.grad(model, argnums=(0,1))
    gradient=grad_loss(J_2x2, s_2x2, ssn_pars, grid_pars, conn_pars, general_pars, label, ref_grating, target_grating, **conv_pars)
    
    return gradient
    #update weights 

In [ ]:

train_data=np.array([10,20])

train_SSN(J_2x2, s_2x2, ssn_pars, grid_pars, conn_pars, general_pars, train_data, stimuli_pars, conv_pars)

In [6]:
conv_pars=dict(dt = 1, xtol = 1e-5, Tmax = 600)
SSN_input_ref =np.matmul(SSN_filters, target_grating.ravel())*A
SSN_input_ref=np.maximum(0, SSN_input_ref)
r_init=np.zeros(SSN_input_ref.shape[0])
fp_ref, _ = ssn.fixed_point_r(SSN_input_ref, r_init=r_init, **conv_pars)

grad_ssn=jax.grad((ssn.fixed_point_r))
grad_ssn(SSN_input_ref, r_init=r_init, **conv_pars)


 Warning 1: reached Tmax=600, before convergence to fixed point.
       max(abs(dx./max(abs(xvec), 1.0))) = 0.1498628854751587,   xtol=1e-05.

Did not reach fixed point.


TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ConcreteArray([2.39045306e-07 5.05174361e-02 3.54168625e+01 1.47604517e+03
 3.53433342e+01 4.93178144e-02 4.55805633e-07 1.40898908e-03
 3.44161719e-01 9.97807801e-05 3.79991978e-01 1.46471069e+03
 2.83114685e+02 1.46274438e+03 3.71743321e-01 3.45970882e-04
 6.38958882e-05 1.96450111e-03 1.36579215e-09 4.94456030e-02
 3.51516914e+01 1.46444324e+03 3.50581055e+01 4.76084426e-02
 3.62756909e-05 2.59627323e-05 1.28611634e-02 1.53637741e-10
 1.10259043e-06 4.41823229e-02 3.57002020e-01 2.58970540e-02
 4.03595768e-05 3.03160959e-06 1.17222185e-03 8.77917588e-01
 1.21548219e-05 1.10309920e-05 9.27108247e-03 4.94957045e-02
 2.59197364e-03 1.15707962e-06 1.18230097e-02 3.12171178e-04
 3.32510471e-03 3.49332261e+00 7.32741091e-06 2.74387912e-06
 1.01096899e-04 5.29806968e-03 3.85003159e-06 5.52617666e-06
 3.25750816e-03 5.01227856e+00 2.41945744e-01 3.02146364e-04
 9.41658072e-

In [ ]:
dt = 1
xtol = 1e-5
Tmax = 600
Nmax = (np.abs((np.round(Tmax/dt))))
np.array(Nmax, int)#

In [ ]:
Nmax